# Data Collector for Earnings Event-Driven RL Trading

This notebook processes raw price data and earnings dates into a structured dataset ready for environment building.

## Overview
1. Load raw price and earnings data
2. Build event windows (N days before and after earnings)
3. Calculate technical features
4. Generate training/test splits
5. Export to CSV format for environment building

## Step 1: Import Libraries and Configuration

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ Libraries imported successfully")

## Step 2: Configuration Parameters

In [ ]:
# ============ Data Paths ============
RAW_PRICE_DATA = "MAG7_hourly_5yr.csv"  # Change this to your file path
EARNINGS_DATA = "mag7_earnings_dates_2019_2024.csv"  # Change this to your file path
OUTPUT_CSV = "earnings_events_data.csv"  # Output file

# ============ Event Window Configuration ============
DAYS_BEFORE = 5      # Days before earnings
DAYS_AFTER = 5       # Days after earnings
TOTAL_WINDOW = DAYS_BEFORE + DAYS_AFTER + 1  # Total days = 11

# ============ Feature Calculation ============
MOMENTUM_WINDOW = 3   # Calculate momentum using past 3 days
VOLATILITY_WINDOW = 3  # Calculate volatility using past 3 days

# ============ Trading Costs ============
TRANSACTION_COST = 0.0005  # 0.05% (round-trip)

# ============ Train/Test Split ============
TRAIN_TEST_SPLIT = 0.80  # 80% training, 20% test

print("Configuration:")
print(f"  Price Data: {RAW_PRICE_DATA}")
print(f"  Earnings Data: {EARNINGS_DATA}")
print(f"  Output CSV: {OUTPUT_CSV}")
print(f"  Event Window: {DAYS_BEFORE} days before + {DAYS_AFTER} days after")
print(f"  Train/Test Split: {TRAIN_TEST_SPLIT:.0%}/{1-TRAIN_TEST_SPLIT:.0%}")

## Step 3: Load Raw Data

In [ ]:
logger.info("Loading raw data...")

# Load price data
price_data = pd.read_csv(RAW_PRICE_DATA)
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
price_data = price_data.sort_values(['ticker', 'timestamp']).reset_index(drop=True)

# Load earnings dates
earnings_dates = pd.read_csv(EARNINGS_DATA)
earnings_dates['earnings_date'] = pd.to_datetime(earnings_dates['earnings_date'])

logger.info(f"✓ Loaded {len(price_data):,} price records")
logger.info(f"✓ Loaded {len(earnings_dates)} earnings release dates")

print("\nPrice Data Sample:")
print(price_data.head())
print(f"\nPrice Data Info:")
print(f"  - Date Range: {price_data['timestamp'].min()} to {price_data['timestamp'].max()}")
print(f"  - Tickers: {price_data['ticker'].unique().tolist()}")
print(f"  - Total Records: {len(price_data):,}")

print("\nEarnings Data Sample:")
print(earnings_dates.head())
print(f"\nEarnings Data Info:")
print(f"  - Date Range: {earnings_dates['earnings_date'].min()} to {earnings_dates['earnings_date'].max()}")
print(f"  - Total Dates: {len(earnings_dates)}")

## Step 4: Build Event Windows

In [ ]:
logger.info("Building event windows...")

all_events_data = []
valid_events_count = 0
skipped_events_count = 0

for _, earnings_row in earnings_dates.iterrows():
    ticker = earnings_row['ticker']
    earnings_date = pd.Timestamp(earnings_row['earnings_date'])
    
    # Get price data for this company
    ticker_data = price_data[price_data['ticker'] == ticker].copy()
    
    # Define event window
    window_start = earnings_date - timedelta(days=DAYS_BEFORE)
    window_end = earnings_date + timedelta(days=DAYS_AFTER)
    
    # Extract window data - convert to date for comparison (avoid timezone issues)
    ticker_data['timestamp_date'] = ticker_data['timestamp'].dt.date
    earnings_start_date = window_start.date()
    earnings_end_date = window_end.date()
    
    window_data = ticker_data[
        (ticker_data['timestamp_date'] >= earnings_start_date) & 
        (ticker_data['timestamp_date'] <= earnings_end_date)
    ].copy()
    
    # Check data completeness
    if len(window_data) < 50:  # Approximately 5 days * 10 hours
        skipped_events_count += 1
        continue
    
    # Sort by timestamp
    window_data = window_data.sort_values('timestamp').reset_index(drop=True)
    window_data = window_data.drop('timestamp_date', axis=1)  # Remove helper column
    
    # Add event metadata
    window_data['ticker_event'] = ticker
    window_data['earnings_date'] = earnings_date
    window_data['window_start'] = window_start
    window_data['window_end'] = window_end
    window_data['event_id'] = valid_events_count
    
    all_events_data.append(window_data)
    valid_events_count += 1

logger.info(f"✓ Built {valid_events_count} valid event windows")
logger.info(f"⊘ Skipped {skipped_events_count} events due to insufficient data")

print(f"\nEvent Windows Summary:")
print(f"  - Valid Events: {valid_events_count}")
print(f"  - Skipped Events: {skipped_events_count}")
print(f"  - Total Data Points in Events: {sum(len(e) for e in all_events_data):,}")

## Step 5: Calculate Technical Features

In [ ]:
def calculate_features(window_data, earnings_date):
    """
    Calculate technical features for an event window.
    
    Returns dict with:
    - momentum: Pre-earnings momentum
    - volatility: Pre-earnings volatility
    - avg_volume: Average volume
    - pre_close: Pre-earnings close price
    - window_return: Window return
    """
    try:
        # Convert earnings_date to date for comparison
        if hasattr(earnings_date, 'date'):
            earnings_date_compare = earnings_date.date()
        else:
            earnings_date_compare = earnings_date
        
        # Separate pre-earnings data
        if 'timestamp_date' in window_data.columns:
            pre_earnings = window_data[window_data['timestamp_date'] < earnings_date_compare]
        else:
            window_data_copy = window_data.copy()
            window_data_copy['temp_date'] = window_data_copy['timestamp'].dt.date
            pre_earnings = window_data_copy[window_data_copy['temp_date'] < earnings_date_compare]
            pre_earnings = pre_earnings.drop('temp_date', axis=1)
        
        if len(pre_earnings) < 20:
            return None
        
        # 1. Pre-earnings momentum (return over past N days)
        pre_returns = pre_earnings['close'].pct_change().dropna()
        momentum = pre_returns.tail(MOMENTUM_WINDOW).sum()
        
        # 2. Volatility (std dev over past N days)
        volatility = pre_returns.tail(VOLATILITY_WINDOW).std()
        
        # 3. Average volume
        avg_volume = pre_earnings['volume'].tail(VOLATILITY_WINDOW).mean()
        
        # 4. Pre-earnings closing price
        pre_close = pre_earnings['close'].iloc[-1]
        
        # 5. Window return (full window)
        pre_price = pre_earnings['close'].iloc[0]
        
        if 'timestamp_date' in window_data.columns:
            post_earnings = window_data[window_data['timestamp_date'] > earnings_date_compare]
        else:
            window_data_copy = window_data.copy()
            window_data_copy['temp_date'] = window_data_copy['timestamp'].dt.date
            post_earnings = window_data_copy[window_data_copy['temp_date'] > earnings_date_compare]
            post_earnings = post_earnings.drop('temp_date', axis=1)
        
        if len(post_earnings) > 0:
            post_price = post_earnings['close'].iloc[-1]
            window_return = (post_price - pre_price) / pre_price
        else:
            window_return = 0
        
        return {
            'momentum': momentum,
            'volatility': volatility if volatility > 0 else 0.01,
            'avg_volume': avg_volume,
            'pre_close': pre_close,
            'window_return': window_return
        }
    
    except Exception as e:
        logger.warning(f"Feature calculation failed: {e}")
        return None

logger.info("Calculating features for all events...")

# Add features to each event
processed_events = []
for event_data in all_events_data:
    ticker = event_data['ticker_event'].iloc[0]
    earnings_date = event_data['earnings_date'].iloc[0]
    
    features = calculate_features(event_data, earnings_date)
    
    if features is not None:
        # Add features to each row in the event
        event_data['momentum'] = features['momentum']
        event_data['volatility'] = features['volatility']
        event_data['avg_volume_event'] = features['avg_volume']
        event_data['pre_close'] = features['pre_close']
        event_data['window_return'] = features['window_return']
        
        processed_events.append(event_data)

logger.info(f"✓ Calculated features for {len(processed_events)} events")

print(f"\nFeature Calculation Summary:")
print(f"  - Events with Features: {len(processed_events)}")
print(f"  - Total Data Points: {sum(len(e) for e in processed_events):,}")

## Step 6: Create Train/Test Split

In [ ]:
# Combine all events into single dataframe
combined_data = pd.concat(processed_events, ignore_index=True)

# Get unique events for train/test split
unique_events = combined_data[['event_id', 'earnings_date']].drop_duplicates('event_id')

# Split events (not individual rows)
train_size = int(len(unique_events) * TRAIN_TEST_SPLIT)
train_event_ids = unique_events.iloc[:train_size]['event_id'].values
test_event_ids = unique_events.iloc[train_size:]['event_id'].values

# Add split label
combined_data['split'] = combined_data['event_id'].apply(
    lambda x: 'train' if x in train_event_ids else 'test'
)

logger.info(f"✓ Train/Test Split:")
logger.info(f"  - Training events: {len(train_event_ids)}")
logger.info(f"  - Test events: {len(test_event_ids)}")

print(f"\nTrain/Test Split Summary:")
print(f"  - Training Events: {len(train_event_ids)}")
print(f"  - Test Events: {len(test_event_ids)}")
print(f"  - Training Rows: {len(combined_data[combined_data['split'] == 'train']):,}")
print(f"  - Test Rows: {len(combined_data[combined_data['split'] == 'test']):,}")

print(f"\nData Sample:")
print(combined_data.head(10))

## Step 7: Feature Statistics

In [ ]:
# Get unique features per event
event_features = combined_data.groupby('event_id').agg({
    'momentum': 'first',
    'volatility': 'first',
    'avg_volume_event': 'first',
    'window_return': 'first',
    'split': 'first'
}).reset_index()

print("\n=== Feature Statistics ===")
print("\nTraining Set Features:")
train_features = event_features[event_features['split'] == 'train']
print(train_features[['momentum', 'volatility', 'window_return']].describe())

print("\nTest Set Features:")
test_features = event_features[event_features['split'] == 'test']
print(test_features[['momentum', 'volatility', 'window_return']].describe())

print(f"\nWindow Return Distribution:")
print(f"  - Positive Returns: {(event_features['window_return'] > 0).sum()} events")
print(f"  - Negative Returns: {(event_features['window_return'] < 0).sum()} events")
print(f"  - Mean Return: {event_features['window_return'].mean():.4f}")
print(f"  - Median Return: {event_features['window_return'].median():.4f}")

## Step 8: Prepare CSV for Environment Building

In [ ]:
# Select relevant columns for environment
csv_columns = [
    'ticker_event',
    'earnings_date',
    'window_start',
    'window_end',
    'event_id',
    'timestamp',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'momentum',
    'volatility',
    'avg_volume_event',
    'pre_close',
    'window_return',
    'split'
]

output_data = combined_data[csv_columns].copy()

# Sort by event and timestamp
output_data = output_data.sort_values(['event_id', 'timestamp']).reset_index(drop=True)

# Save to CSV
output_data.to_csv(OUTPUT_CSV, index=False)

logger.info(f"✓ Saved CSV to {OUTPUT_CSV}")

print(f"\n✅ CSV Export Complete!")
print(f"\nFile: {OUTPUT_CSV}")
print(f"\nColumns:")
for i, col in enumerate(csv_columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nDataset Size:")
print(f"  - Total Rows: {len(output_data):,}")
print(f"  - Total Columns: {len(csv_columns)}")
print(f"  - File Size: {Path(OUTPUT_CSV).stat().st_size / 1024:.2f} KB")

print(f"\nFirst Few Rows:")
print(output_data.head())

print(f"\nLast Few Rows:")
print(output_data.tail())

## Step 9: Data Quality Check

In [ ]:
print("\n=== Data Quality Check ===")

# Check for missing values
missing_values = output_data.isnull().sum()
print(f"\nMissing Values:")
for col in missing_values[missing_values > 0].index:
    print(f"  - {col}: {missing_values[col]}")
if missing_values.sum() == 0:
    print("  ✓ No missing values")

# Check for duplicates
duplicates = output_data.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates}")

# Check split distribution
print(f"\nSplit Distribution:")
print(output_data['split'].value_counts())

# Check data types
print(f"\nData Types:")
print(output_data.dtypes)

# Check for outliers
print(f"\nPrice Range Check:")
print(f"  - Close Price Min: ${output_data['close'].min():.2f}")
print(f"  - Close Price Max: ${output_data['close'].max():.2f}")
print(f"  - Volume Min: {output_data['volume'].min():,}")
print(f"  - Volume Max: {output_data['volume'].max():,}")

print(f"\n✅ Data Quality Check Complete!")

## Step 10: Usage Instructions for Environment Building

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════════════════════╗
║                    CSV Data Ready for Environment Building                       ║
╚════════════════════════════════════════════════════════════════════════════════════╝

📁 Output File: earnings_events_data.csv

📋 How to Use in Environment:

1. Load the CSV:
   ```python
   import pandas as pd
   df = pd.read_csv('earnings_events_data.csv')
   ```

2. Get unique events:
   ```python
   events = df.groupby('event_id').first().reset_index()
   train_events = events[events['split'] == 'train']
   test_events = events[events['split'] == 'test']
   ```

3. For each event, create environment:
   ```python
   event_data = df[df['event_id'] == event_id]
   env = EarningsEventEnv({
       'ticker': event_data['ticker_event'].iloc[0],
       'earnings_date': event_data['earnings_date'].iloc[0],
       'price_data': event_data[['timestamp', 'open', 'high', 'low', 'close', 'volume']],
       'features': {
           'momentum': event_data['momentum'].iloc[0],
           'volatility': event_data['volatility'].iloc[0],
           'avg_volume': event_data['avg_volume_event'].iloc[0],
           'pre_close': event_data['pre_close'].iloc[0],
           'window_return': event_data['window_return'].iloc[0]
       }
   })
   ```

📊 CSV Columns Description:
   - ticker_event: Stock ticker symbol
   - earnings_date: Earnings announcement date
   - window_start: Event window start date
   - window_end: Event window end date
   - event_id: Unique event identifier
   - timestamp: Price timestamp (hourly)
   - open/high/low/close: OHLC prices
   - volume: Trading volume
   - momentum: Pre-earnings momentum
   - volatility: Pre-earnings volatility
   - avg_volume_event: Average volume in event window
   - pre_close: Closing price before earnings
   - window_return: Return from start to end of window
   - split: Train/Test split (train/test)

✅ Ready to build environments!
""")

## Summary Statistics

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\n📊 Dataset Statistics:")
print(f"  - Total Events: {output_data['event_id'].nunique()}")
print(f"  - Training Events: {len(output_data[output_data['split'] == 'train']['event_id'].unique())}")
print(f"  - Test Events: {len(output_data[output_data['split'] == 'test']['event_id'].unique())}")
print(f"  - Total Data Points: {len(output_data):,}")
print(f"  - Unique Tickers: {output_data['ticker_event'].nunique()}")
print(f"  - Date Range: {output_data['timestamp'].min()} to {output_data['timestamp'].max()}")

print(f"\n💾 Output File:")
print(f"  - Filename: {OUTPUT_CSV}")
print(f"  - Size: {Path(OUTPUT_CSV).stat().st_size / (1024*1024):.2f} MB")
print(f"  - Format: CSV (can be opened in Excel, pandas, etc.)")

print(f"\n✅ Data collection complete! Ready for environment building.")
print("="*80)